In [1]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import re
from urllib.parse import urlparse
from pathlib import Path

In [2]:
import sys
sys.path.append(Path().cwd().parent.as_posix())

from hospwork import get_work_page
from hospwork import findjobtype

In [3]:
g=Path('.')
g.cwd().parent.joinpath('cacert.pem')

PosixPath('/home/dvdd/hospwork/cacert.pem')

In [4]:
url_base='https://www.vhcy.gov.tw'
url_work_table='/UnitPage/RowView?WebMenuID=fb60be6b-ced8-485b-95dc-b470a3c4264f&UnitID=1f7b14c3-842f-4091-b3ca-4972e4c53524%20&UnitDefaultTemplate=1'
url=url_base+url_work_table
g=requests.get(url, verify=Path().cwd().parent.joinpath('cacert.pem'))
soup=BeautifulSoup(g.content, 'html.parser')

# Get page link
In this website the work table has some page, so the function should find page link for next step.

In [5]:
pages_link=[]
pages_link.append(url)
for o in soup.find("div",class_="pager").find_all("a"):
    if o.text == "下一頁" or o.text == "»":
        pass
    else:
        link = url_base+o.get('href')
        pages_link.append(link)
        print(o.text,link)

2 https://www.vhcy.gov.tw/UnitPage/RowView?UnitID=1f7b14c3-842f-4091-b3ca-4972e4c53524&UnitDefaultTemplate=1&WebMenuID=fb60be6b-ced8-485b-95dc-b470a3c4264f&page=2
3 https://www.vhcy.gov.tw/UnitPage/RowView?UnitID=1f7b14c3-842f-4091-b3ca-4972e4c53524&UnitDefaultTemplate=1&WebMenuID=fb60be6b-ced8-485b-95dc-b470a3c4264f&page=3
4 https://www.vhcy.gov.tw/UnitPage/RowView?UnitID=1f7b14c3-842f-4091-b3ca-4972e4c53524&UnitDefaultTemplate=1&WebMenuID=fb60be6b-ced8-485b-95dc-b470a3c4264f&page=4
5 https://www.vhcy.gov.tw/UnitPage/RowView?UnitID=1f7b14c3-842f-4091-b3ca-4972e4c53524&UnitDefaultTemplate=1&WebMenuID=fb60be6b-ced8-485b-95dc-b470a3c4264f&page=5
6 https://www.vhcy.gov.tw/UnitPage/RowView?UnitID=1f7b14c3-842f-4091-b3ca-4972e4c53524&UnitDefaultTemplate=1&WebMenuID=fb60be6b-ced8-485b-95dc-b470a3c4264f&page=6
7 https://www.vhcy.gov.tw/UnitPage/RowView?UnitID=1f7b14c3-842f-4091-b3ca-4972e4c53524&UnitDefaultTemplate=1&WebMenuID=fb60be6b-ced8-485b-95dc-b470a3c4264f&page=7


# Get work table

The `get_work_page` function is used for getting each work page

In [6]:
work_page = soup.find("tbody")

In [7]:
def get_work_detail(link):
    work_detail = get_work_page(link, verify=Path().cwd().parent.joinpath('cacert.pem')).find("div",class_="newContent").text
    if work_detail.replace("\n","") == "":
        return None
    else:
        return get_work_page(link, verify=Path().cwd().parent.joinpath('cacert.pem')).find("div",class_="newContent").text.replace("\r","").replace("\t","").split("\n")

In [8]:
def get_work_title(title):
    try:
        new_title = re.search("\B[科,短,社,契,牙,秘,醫,部,室](.*)[員,理,工,師,生]",title).group(0)
    except AttributeError:
        print("ERROR:",title)
    return new_title

In [9]:
def get_work_originazation(title,link):
    try:
        origination = re.search("((.*)[組,部,科,室,心])",title).group(1)
    except AttributeError:
        g = get_work_detail(link)
        if g == None:
            return "check page"
        try:
            origination = [gg for gg in g if "職稱" in gg][0].split("：")[-1]
        except AttributeError:
            ggg = [gg for gg in g if gg != '']
            origination = ggg[ggg.index("職稱")+1]
        except:
            origination = "check page"
    return origination

In [10]:
def get_work_deadtime(link):
    g = get_work_detail(link)
    if g == None:
        return "check page"
    try:
        ggg = [gg for gg in g if gg != '']
        try:
            dead = re.findall("\d+年\d+月\d+日|\d+.\d+.\d+",ggg[ggg.index("上網期間")+1])[-1]
        except:
            for gggg in ggg:
                
                if "上網期間" in gggg or "報名方式" in gggg or "報名人員請於" in gggg or "日前" in gggg or "報名日期" in gggg or "（含）前" in gggg:
                    try:
                        dead = re.findall("\d+年\d+月\d+日|\d+.\d+.\d+",[gg for gg in g if "上網期間" in gg or "報名方式" in gg or "報名日期" in gg or "（含）前" in gg][0].split("：")[-1])[-1]
                    except:
                        dead = "check page"
    except:
        dead = "check page"
    
    finally:
        if 'dead' not in locals():
            dead = "check page"
        return dead

In [11]:
def get_work_table(table_,work_table):
    for p_i, p_item in enumerate(table_.find_all("tr")):
        if p_item.find('a'):
            p_item_a = p_item.find('a')
            title = p_item_a.text
            if "錄取公告" in title or '考試公告' in title or "甄試結果公告" in title or "核定" in title:
                break
            new_title = findjobtype(title,'臺中榮民總醫院嘉義分院').replace("部契約","契約")
            link_s = url_base+p_item_a.get('href')
            if get_work_detail(link_s) == None:
                break
            origination = get_work_originazation(title,link_s)
            try:
                dead_line = get_work_deadtime(link_s)
            except:
                print("Error",p_item)
            print(p_i-1,new_title,link_s,dead_line, origination)
            work_table.append([new_title, dead_line, origination, link_s ])
    return work_table

In [12]:
work_table=[]
work_table = get_work_table(work_page,work_table)
for p_i, p_item in enumerate(pages_link):
    table_ = get_work_page(p_item, verify=Path().cwd().parent.joinpath('cacert.pem'))

    work_table = get_work_table(table_,work_table)

-1 契約居家護理 https://www.vhcy.gov.tw/UnitPage/RowViewDetail?WebRowsID=78f932dd-dce7-41e7-9145-e4fe2d4c65a7&UnitID=1f7b14c3-842f-4091-b3ca-4972e4c53524&CompanyID=e8e0488e-54a0-44bf-b10c-d029c423f6e7&UnitDefaultTemplate=1 113年7 精神部
0 契約社工師( https://www.vhcy.gov.tw/UnitPage/RowViewDetail?WebRowsID=fd1324d8-0010-49b2-914a-8a0a967fd1b9&UnitID=1f7b14c3-842f-4091-b3ca-4972e4c53524&CompanyID=e8e0488e-54a0-44bf-b10c-d029c423f6e7&UnitDefaultTemplate=1 check page 社會工作室
1 學部(健檢中心)契約辦事員1名徵才公告(職務代 https://www.vhcy.gov.tw/UnitPage/RowViewDetail?WebRowsID=6ecf9841-836b-4f70-be13-ceae70eb5464&UnitID=1f7b14c3-842f-4091-b3ca-4972e4c53524&CompanyID=e8e0488e-54a0-44bf-b10c-d029c423f6e7&UnitDefaultTemplate=1 check page 家庭醫學部(健檢中心
2 (灣橋分院)社區整體照顧服務體系A級單位個管 https://www.vhcy.gov.tw/UnitPage/RowViewDetail?WebRowsID=489486de-c5a8-432b-9b7f-ea92cdb66677&UnitID=1f7b14c3-842f-4091-b3ca-4972e4c53524&CompanyID=e8e0488e-54a0-44bf-b10c-d029c423f6e7&UnitDefaultTemplate=1 113年08月30日 護理部
3 契約物理治療 https://www.vhcy.gov.tw/UnitP

Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.


AttributeError: 'NoneType' object has no attribute 'text'

In [13]:
work_table=pd.DataFrame(work_table, columns=['召聘職稱','期限' ,"職缺單位" ,'連結'])

print(work_table)

                              召聘職稱          期限                    職缺單位  \
0                           契約居家護理       113年7                     精神部   
1                           契約社工師(  check page                   社會工作室   
2          學部(健檢中心)契約辦事員1名徵才公告(職務代  check page              家庭醫學部(健檢中心   
3           (灣橋分院)社區整體照顧服務體系A級單位個管  113年08月30日                     護理部   
4                           契約物理治療   113年8月10日                     復健科   
5                      (醫療費用組)契約辦事  113年07月31日             醫務企管室(醫療費用組   
6       嘉義縣113年度輔具資源中心徵求維修人員1名徵才公告   113年8月31日          嘉義縣113年度輔具資源中心   
7                       部(胃鏡室)契約護理   113年7月22日                 內科部(胃鏡室   
8                   部(胃鏡室)契約部分工時護理       113年7             內科部(胃鏡室)契約部   
9                           契約居家護理       113年7                     精神部   
10                          契約社工師(  check page                   社會工作室   
11         學部(健檢中心)契約辦事員1名徵才公告(職務代  check page              家庭醫學部(健檢中心   
12          (灣橋分院)社區整體照顧服務體系A級單位個管  11

In [145]:
get_work_page('https://www.vhcy.gov.tw//PageView/RowViewDetail?WebRowsID=2296c3cc-ea34-4273-beac-603a99803d97&amp;UnitID=fbb1c734-f7d0-41c4-93d8-261f32f1db04&amp;CompanyID=E8E0488E-54A0-44BF-B10C-D029C423F6E7').find("div",class_="newContent").text

'\n\n'